In [1]:
import time
import random
import torch
import torch.nn as nn
import torch.optim as optim
import sentencepiece as spm
import torch.nn.functional as F

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


en_model_prefix = "en_multi30k_word"
de_model_prefix = "de_multi30k_word"

en_sp = spm.SentencePieceProcessor()
de_sp = spm.SentencePieceProcessor()

en_sp.Load(f"{en_model_prefix}.model")
de_sp.Load(f"{de_model_prefix}.model")

word_vec_size = 512
en_vocab_size = en_sp.GetPieceSize()
de_vocab_size = de_sp.GetPieceSize()
en_word_padding_idx = en_sp.pad_id()
de_word_padding_idx = de_sp.pad_id()
dropout = 0.1
position_encoding = True
MAX_LENGTH = 100

def numericalize(text, tokenizer):
    ids = tokenizer.EncodeAsIds(text)
    return torch.tensor(ids)

def pad_sequences(src, tgt):
    max_length = max(max(len(s) for s in src), max(len(t) for t in tgt))

    # Pad the sequences to the maximum length
    src_padded = [
        torch.cat([s, torch.tensor([en_word_padding_idx] * (max_length - len(s)), dtype=torch.long)])
        for s in src
    ]
    tgt_padded = [
        torch.cat([t, torch.tensor([de_word_padding_idx] * (max_length - len(t)), dtype=torch.long)])
        for t in tgt
    ]

    # Convert to tensors
    src_padded = torch.stack(src_padded)
    tgt_padded = torch.stack(tgt_padded)
    

    return src_padded, tgt_padded

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [4]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=DEVICE).fill_(de_sp.bos_id())
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(encoder_outputs.shape[1]):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [5]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=DEVICE).fill_(de_sp.bos_id())
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(encoder_outputs.shape[1]):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input


        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [6]:
def train_en_to_de(
    encoder,
    decoder,
    en_sp,
    de_sp,
    learning_rate=0.001,
    num_epochs=10,
    batch_size=32,
):
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss().to(DEVICE)

    def load_data(en_filepath, de_filepath):
        en_data = []
        de_data = []
        with open(en_filepath, "r", encoding="utf-8") as en_f, open(
            de_filepath, "r", encoding="utf-8"
        ) as de_f:
            for en_line, de_line in zip(en_f, de_f):
                en_data.append(en_line.strip())
                de_data.append(de_line.strip())
        return en_data, de_data

    en_train, de_train = load_data("multi30k_train_en.txt", "multi30k_train_de.txt")

    for epoch in range(num_epochs):
        start_time = time.time()
        total_loss = 0

        data = list(zip(en_train, de_train))
        random.shuffle(data)
        en_train, de_train = zip(*data)

        for i in range(0, len(en_train), batch_size):
            en_batch = en_train[i : i + batch_size]
            de_batch = de_train[i : i + batch_size]

            # Numericalize and pad the *entire batch*
            en_numericalized = [numericalize(text, en_sp) for text in en_batch]
            de_numericalized = [numericalize(text, de_sp) for text in de_batch]


            ip, op = pad_sequences(en_numericalized, de_numericalized)

            input_tensor = ip.to(DEVICE)
            target_tensor = op.to(DEVICE)
            
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()

            encoder_outputs, encoder_hidden = encoder(input_tensor)

            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)), target_tensor.view(-1)
            )
            loss.backward()

            encoder_optimizer.step()
            decoder_optimizer.step()

            total_loss += (loss.item() / len(en_train))

        end_time = time.time()
        epoch_time = end_time - start_time

        print(
            f"Epoch: {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Time: {epoch_time:.2f}s"
        )

In [7]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [8]:
def evaluate(encoder, decoder, sentence, en_sp, de_sp):
    with torch.no_grad():
        input_tensor = numericalize(sentence, en_sp).view(1, -1)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == de_sp.eos_id():
                decoded_words.append('<EOS>')
                break
            decoded_words.append(de_sp.IdToPiece(idx.item()))
    return decoded_words, decoder_attn

In [9]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = ["a young man in a black and yellow jacket is gazing at something and smiling .", "ein junger mann in einer schwarz-gelben jacke blickt etwas an und lächelt ."]
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], en_sp, de_sp)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
encoder = EncoderRNN(en_vocab_size, hidden_size=512).to(DEVICE)
decoder = AttnDecoderRNN(hidden_size=512, output_size=de_vocab_size).to(DEVICE)

train_en_to_de(encoder, decoder, en_sp, de_sp)

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, en_sp, de_sp)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')